In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
!nvidia-smi

Tue Jan  2 09:31:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   36C    P8              26W / 350W |      3MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/" #"/content/drive/MyDrive/persuasion_technique_detection/"

In [3]:
#!pip install keras-ocr --quiet
#!pip install tensorflow
#!pip install keras

In [4]:
import json
import os
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import math
import numpy as np
import pandas as pd

# Read data

In [5]:
val_path="data/subtask2b/val.json"
train_path="data/subtask2b/train.json"
test_path="data/subtask2b/dev_unlabeled.json"

with open(folder_name+val_path) as f:
  d = json.load(f)
  val=pd.DataFrame.from_dict(d)
  labels=val["label"]
  num_label=[int(el=="propagandistic") for el in labels]
  val["num_label"]=num_label
  val=val.drop(columns=['label'])
  val_set=val.rename(columns={"num_label": "label"})

with open(folder_name+train_path) as f:
  d = json.load(f)
  train=pd.DataFrame.from_dict(d)
  labels=train["label"]
  num_label=[int(el=="propagandistic") for el in labels]
  train["num_label"]=num_label
  train=train.drop(columns=['label'])
  train_set=train.rename(columns={"num_label": "label"})
  #mask = train_set['image'] == "prop_meme_24871.png"
  #train_set = train_set[~mask]

with open(folder_name+test_path) as f:
  d = json.load(f)
  dev_unlabeled_set=pd.DataFrame.from_dict(d)

label2num={"non_propagandistic":0,"propagandistic":1}
num2label={0:"non_propagandistic",1:"propagandistic"}

print(len(train_set),len(val_set),len(dev_unlabeled_set))


1199 150 300


# Remove texts from images and save processed images
Source: https://towardsdatascience.com/remove-text-from-images-using-cv2-and-keras-ocr-24e7612ae4f4

In [33]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2) / 2)
    y_mid = int((y1 + y2) / 2)
    return (x_mid, y_mid)

def inpaint_text(input_path, output_path, images, pipeline):
    for image_name in images:
        try:
          img_path = os.path.join(input_path, image_name)  # Use os.path.join for path concatenation
          img = keras_ocr.tools.read(img_path)
          img_shape = img.shape
          
          prediction_groups = pipeline.recognize([img])
          mask = np.zeros(img.shape[:2], dtype="uint8")
          for box in prediction_groups[0]:
              x0, y0 = box[1][0]
              x1, y1 = box[1][1]
              x2, y2 = box[1][2]
              x3, y3 = box[1][3]

              x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
              x_mid1, y_mid1 = midpoint(x0, y0, x3, y3)
              thickness = int(math.sqrt((x2 - x1)**2 + (y2 - y1)**2))
              cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255, thickness)
          # Ensure the mask has the same dimensions as the image
          mask = cv2.resize(mask, (img.shape[1], img.shape[0]))
          img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
          img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          if not os.path.exists(output_path):
              os.makedirs(output_path)

          cv2.imwrite(os.path.join(output_path, image_name), img_rgb)
        except Exception as e:
          print(e,image_name)
          continue

In [ ]:
pipeline = keras_ocr.pipeline.Pipeline()
input_path=folder_name+"data/subtask2b/subtask2b_images/train/"
output_path=folder_name+"data/subtask2b/subtask2b_images_textsRemoved/train/"
inpaint_text(input_path,output_path,train_set["image"],pipeline)
input_path=folder_name+"data/subtask2b/subtask2b_images/val/"
output_path=folder_name+"data/subtask2b/subtask2b_images_textsRemoved/val/"
inpaint_text(input_path,output_path,val_set["image"],pipeline)
input_path=folder_name+"data/subtask2b/subtask2b_images/dev/"
output_path=folder_name+"data/subtask2b/subtask2b_images_textsRemoved/dev/"
inpaint_text(input_path,output_path,dev_unlabeled_set["image"],pipeline)

Looking for /home/nlp-lab-ws23/.keras-ocr/craft_mlt_25k.h5
Looking for /home/nlp-lab-ws23/.keras-ocr/crnn_kurapan.h5


In [6]:
original=set(os.listdir(folder_name+"data/subtask2b/subtask2b_images/train/"))
removed=set(os.listdir(folder_name+"data/subtask2b/subtask2b_images_textsRemoved/train/"))
print(original-removed)

{'prop_meme_2758.png', 'prop_meme_2723.png', '.ipynb_checkpoints'}


In [8]:
original=set(os.listdir(folder_name+"data/subtask2b/subtask2b_images/val/"))
removed=set(os.listdir(folder_name+"data/subtask2b/subtask2b_images_textsRemoved/val/"))
print(original-removed)

set()


In [9]:
original=set(os.listdir(folder_name+"data/subtask2b/subtask2b_images/dev/"))
removed=set(os.listdir(folder_name+"data/subtask2b/subtask2b_images_textsRemoved/dev/"))
print(original-removed)

set()
